# Bilibili Scraping  - 弹幕

In [24]:
# import library
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import datetime
import time 
import xml.etree.ElementTree as etree
import re

In [ ]:
https://api.bilibili.com/x/v1/dm/list.so?oid=45508213

In [133]:
def get_danmaku(aid,df):
    # obtain cid 
    url_cid = 'https://www.bilibili.com/widget/getPageList?aid='+ str(aid)
    response_cid = requests.get(url_cid)
    json_cid = json.loads(response_cid.text)
    cid = json_cid[0]['cid']
    
    url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
    response = requests.get(url)
    html_doc = response.content.decode('utf-8')
    # html_doc = <d p="54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602">我狂铁胜率百分之85</d>
    format_doc = re.compile('<d p="(.*?)">(.*?)</d>')
    # danmu = [(('54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602','我狂铁胜率百分之85')),()]
    # () = tuple
    danmu = format_doc.findall(html_doc)
    
    # function of organize data
    def reformat(tuples):
        # info = '54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602' - str
        info = tuples[0]
        # words = '我狂铁胜率百分之85'
        words = tuples[1]
        # info = [54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602] - list
        # info = [弹幕出现的时间,弹幕的模式]
        info = info.split(',')
        # timestamp, readable datatime 
        #return [[['1531313705', '2018-07-11 12:55:05', '我狂铁胜率百分之85']],[],[]]
        return [info[4],str(pd.to_datetime(info[4],unit='s')),words]
    
    map_f = map(reformat, danmu)
    data=list(map_f)
    
    df_new = pd.DataFrame(data,columns=['dm_timestamp', 'dm_date', 'dm_content'])
    df_new['aid'] = str(aid)
    df_new['cid'] = str(cid)
    df_new = df_new[['aid','cid','dm_timestamp', 'dm_date', 'dm_content']]

    df = pd.concat([df, df_new], axis=0, sort=False)
    return df_new

df_new = get_danmaku('455082139',df)
df_new.head(2)

,aid,cid,dm_timestamp,dm_date,dm_content
0,455082139,171794896,1585579760,2020-03-30 14:49:20,刚刚
1,455082139,171794896,1585579773,2020-03-30 14:49:33,……


54.01300,1,25,16777215,1531313705,0,f1d4f711,2187083291033602  
54.01300    : 弹幕出现的时间: 以秒为单位  
1           : 弹幕的模式: 1～3 滚动弹幕 4 底端弹幕 5 顶端弹幕 6 逆向弹幕 7 精准定位 8 高级弹幕  
25          : 字号: 12 非常小 16 特小 18 小 25 中 36 大 45 很大 64 特别大  
16777215    : 字体的颜色：将 HTML 六位十六进制颜色转为十进制表示，例如 #FFFFFF 会被存储为 16777215
1531313705  :时间戳: 以毫秒为单位，基准时间为 1970-1-1 08:00:00  
0           :弹幕池: 0 普通池 1 字幕池 2 特殊池（注：目前特殊池为高级弹幕专用）  
f1d4f711    :发送者的 ID: 用于「屏蔽此弹幕的发送者」功能  
2187083291033602 : 弹幕在弹幕数据库中 rowID，用于「历史弹幕」功能  

,aid,cid,dm_timestamp,dm_date,dm_content
0,455082139,171794896,1585579760,2020-03-30 14:49:20,刚刚
1,455082139,171794896,1585579773,2020-03-30 14:49:33,……
2,455082139,171794896,1585579840,2020-03-30 14:50:40,你还不戴口罩
3,455082139,171794896,1585579842,2020-03-30 14:50:42,好早啊
4,455082139,171794896,1585579857,2020-03-30 14:50:57,。。。
5,455082139,171794896,1585579866,2020-03-30 14:51:06,美国人民受苦了
6,455082139,171794896,1585579880,2020-03-30 14:51:20,他在狂笑
7,455082139,171794896,1585579884,2020-03-30 14:51:24,所以说来说去你还是不戴口罩
8,455082139,171794896,1585579919,2020-03-30 14:51:59,私信来的就是硬气
9,455082139,171794896,1585579942,2020-03-30 14:52:22,靠，灾难片既视感


In [132]:
aid_list_df = pd.read_csv('共青团视频(11.20-3.20)(as of 4.1).csv')
aid_list_df.aid

0      98074209
1      97931499
2      97584279
3      97390925
4      97212672
5      97122823
6      97109018
7      96810325
8      96762566
9      96574512
10     96488000
11     95885550
12     95877065
13     95717598
14     95410338
15     95237292
16     95236484
17     94980631
18     94924222
19     94911371
20     94673985
21     94459415
22     94548443
23     94547492
24     94371400
25     94153903
26     93975554
27     93947227
28     93862087
29     93583884
         ...   
184    78778449
185    78676686
186    78562932
187    78309491
188    78266046
189    78108476
190    78107150
191    78108022
192    78105054
193    77975146
194    77929401
195    77806424
196    77363037
197    77303301
198    77298289
199    77290655
200    77201944
201    77087288
202    77107340
203    76947515
204    76877449
205    76838810
206    76775549
207    76450224
208    76558189
209    76524390
210    76485537
211    76467274
212    76410258
213    76394931
Name: aid, Length: 214, 

In [134]:
df = pd.DataFrame(columns=['dm_timestamp', 'dm_date', 'dm_content'])

for i in list(aid_list_df.aid):
    df_new = get_danmaku(i,df)
    df = pd.concat([df_new,df], axis=0, sort=False)
df.head()

,aid,cid,dm_timestamp,dm_date,dm_content
0,76394931,130684654,1583161626,2020-03-02 15:07:06,广东粗口
1,76394931,130684654,1583163671,2020-03-02 15:41:11,潮汕脏话前来报道
2,76394931,130684654,1583163694,2020-03-02 15:41:34,潮汕脏话
3,76394931,130684654,1583163705,2020-03-02 15:41:45,草
4,76394931,130684654,1583163716,2020-03-02 15:41:56,扑领姨


In [135]:
df.shape

(143147, 5)

In [137]:
df.to_csv('共青团视频弹幕(11.20-3.20)(数据截止 4.1).csv')

In [150]:
df.to_excel('共青团视频弹幕(11.20-3.20)(数据截止 4.1).xlsx')